I'll revise the document to focus only on employment and self-employment income:

# Tax-free childcare

[Official government guidance](https://www.gov.uk/tax-free-childcare?step-by-step-nav=d78aeaf6-1747-4d72-9619-f16efb4dd89d) | [Legislation](https://www.legislation.gov.uk/ukdsi/2015/9780111127063)

## Program description

### Overview

Tax-free childcare helps working families with childcare costs. For every **£8** paid into a childcare account, the government adds **£2**.

The program provides financial support up to **£2,000** per year (**£500** per quarter) for each eligible child. For disabled children, this support increases to **£4,000** per year (**£1,000** per quarter).

### Eligibility requirements

#### Age criteria
Standard childcare support extends until September 1st following the child's **11th** birthday. For **disabled children** who receive disability benefits or are certified as **blind/severely sight-impaired**, support continues until **September 1st** following their **16th** birthday.

#### Income requirements
The program considers employment and self-employment income. The maximum adjusted net income allowed is **£100,000** per year for each partner. Minimum earnings requirements (per **3 months**) vary by age:

| Age group | Minimum earnings |
|-----------|-----------------|
| **21+** | **£2,379** |
| **18-20** | **£1,788** |
| **Under 18/Apprentices** | **£1,331** |

#### Work status
Eligible individuals must be either **employed**, **self-employed**, or on qualifying leave (**sick**, **annual**, or **shared parental leave**). Applications can be submitted if starting work within **31 days**.

Non-working partners may still qualify if they receive any of these benefits: **incapacity benefit**, **severe disablement allowance**, **carer's allowance**, or **contribution-based employment and support allowance**.

### Additional rules

#### Income assessment
The program considers:
* Employment income
* Self-employment income
* **New self-employed** businesses (under **12 months**) benefit from more flexible earnings requirements

#### Program interactions
Tax-free childcare is compatible with the **15/30 hours free childcare** program but cannot be combined with:
* **Working tax credit**
* **Child tax credit**
* **Universal credit**
* **Childcare vouchers**

When approved for tax-free childcare:
* Existing benefits terminate automatically
* Childcare vouchers must be cancelled within **90 days**

## Implementation

In the following steps, we explain how PolicyEngine models the Tax-Free Childcare program through parameter definitions and calculations.

### 1. Age determination
First, we determine which children in the household are age-eligible for the program, as Tax-Free Childcare is only available until specific age thresholds.
* `standard.yaml` and `disability.yaml` define age thresholds
* `tax_free_childcare_child_age_eligible.py` calculates eligibility by:
  - Checking person's age
  - Determining disability/blind status
  - Applying appropriate age limit (standard or disability-based)
  - Returns boolean indicating if age requirement is met

Example table:

| Child age | Is disabled | Is age eligible |
|-----------|-------------|-----------------|
| 10        | False       | True            |
| 12        | False       | False           |
| 15        | True        | True            |
| 17        | True        | False           |

### 2. Income assessment
After identifying eligible children, we evaluate if the parents meet the income criteria, examining both minimum and maximum thresholds.
* `countable_sources.yaml` specifies eligible income types
* `income_limit.yaml` sets the maximum income threshold
* `minimum_weekly_hours.yaml` defines weekly work requirements
* `tax_free_childcare_meets_income_requirements.py` calculates:
  - Annual eligible income from countable sources
  - Quarterly income requirements (income / 4)
  - Minimum threshold based on minimum wage and required hours
  - Compliance with income limits using adjusted net income

Example table:

| Adult age | Quarterly earnings | Is over £100k | Is income eligible |
|-----------|-------------------|---------------|-------------------|
| 22        | £3,000            | False         | True              |
| 19        | £1,500            | False         | False             |
| 35        | £10,000           | True          | False             |
| 25        | £4,000            | False         | True              |

### 3. Program compatibility
We check if the household receives any benefits that would make them ineligible for Tax-Free Childcare.
* `disqualifying_benefits.yaml` lists incompatible benefits
* `tax_free_childcare_program_eligible.py`:
  - Checks person's benefit unit for disqualifying benefits
  - Returns true if no incompatible benefits are received

Example table:

| Receives working tax credit | Receives child tax credit | Receives universal credit | Is eligible for program |
|----------------------------|---------------------------|-------------------------|----------------------|
| False                      | False                     | False                   | True                 |
| True                       | False                     | False                   | False                |
| False                      | True                      | False                   | False                |
| False                      | False                     | True                    | False                |

### 4. Work status
The program requires parents to be working, with special provisions for couples where one partner has a disability.
* `tax_free_childcare_work_condition.py` evaluates:
  - Single adults: must be working
  - Couples: either both working or one working with partner having disability
  - Disability status based on relevant benefit receipt or incapacity benefit
  - Returns boolean indicating if work requirements are met

Example table:

| Family composition | Is partner 1 working | Is partner 2 working | Is partner 2 disabled | Meets work condition |
|-------------------|---------------------|---------------------|---------------------|-------------------|
| Single            | True                | N/A                 | N/A                 | True              |
| Single            | False               | N/A                 | N/A                 | False             |
| Couple            | True                | True                | False               | True              |
| Couple            | False               | False               | True                | False             |

### 5. Final eligibility
We combine all previous eligibility checks to determine overall household qualification.
* `tax_free_childcare_eligible.py` checks:
  - At least one child meets age requirements
  - All adults meet income requirements (except children)
  - No disqualifying benefits are received
  - Work conditions are satisfied
  - Uses logical AND to combine all conditions

Example table:

| Is age eligible | Is income eligible | Is program eligible | Meets work condition | Is eligible overall |
|----------------|-------------------|-------------------|-------------------|------------------|
| True           | True              | True               | True              | True             |
| False          | True              | True               | True              | False            |
| True           | False             | True               | True              | False            |

### 6. Contribution calculation
For eligible households, we calculate the government contribution amount.
* `tax_free_childcare.py`:
  - Identifies disabled/blind status for each child
  - Applies appropriate rate (standard or higher) based on disability status
  - Caps contribution based on actual childcare expenses
  - Sums total contribution across all eligible children in the benefit unit

Example table:

| Is eligible | Is child disabled | Annual childcare expense | Annual government contribution |
|------------|------------------|------------------------|---------------------------|
| True       | False            | £8,000                 | £2,000                    |
| True       | True             | £8,000                 | £4,000                    |
| False      | False            | £8,000                 | £0                        |
| True       | True             | £12,000                | £4,000                    |
